## What to Expect From This Notebook
- This notebook is a demonstration of the proper way to calculate the number of pixels to project a satellite image to when given latitude and longitude or path and row WRS-2 coordinates
- This notebook takes into account the geoid shape of Earth using the latest World Geodetic System (WGS84)

## Algorithmic Steps
- Turn path and row WRS coordinates into latitude and longitude coordinates if not already
- Determine the number of meters per degree of latitude and longitude
- Determine the pixel count of one degree of latitude and longitude using the meters per degree and satellite image resolution
- Calculate the tile resolution that should be used using meters per degree and satellite image resolution
- Determine tile size in pixels using tile resolution and meters per degree of latitude and longitude

![](diagrams/ingestion/wrs2.gif)  
***
## How It Works
To determine the appropriate pixel size to use for a geographic region there are three basic factors to take into account:
  - <font color=green>The resolution of the satellite</font>
    - Landsat satellites have a resolution of 30m per pixel
  - <font color=green>The latitude</font>
    - The longitude does not matter.  The meters per degree of longitude are determined by the latitude since the longitudinal lines converge at the poles and reach their maxima at the equator. 
  - <font color=green>The actual shape of the Earth</font>
    - Earth is not a true sphere, it is actually a geoid.  Becuase of this we have to use a gravimetrically oriented ellipsoid to approximate Earth.
    - We will use the WGS84 measurements of the Earth's semimajor and semiminor axes since that is what GPS relies on and is very accurate.  It is estimated that the true center of Earth's mass lies within 2cm of the WGS84 measurements.
![](diagrams/ingestion/geodesic_sphere.png)  

In [1]:
import math
import pandas as pd
import requests
import time

#Getter for Epoch time
current_milli_time = lambda: int(round(time.time() * 1000))

## What are WRS-2 Path and Rows?

USGS satellites follow tilted, nearly vertical paths, when compared to Earth's poles.  These paths are geodesic, one full cycle does not end where it starts.
- <font color=purple>Paths</font> are the intuitive name given to the vertical rows of the geodesic paths taken by the satellites.
    - This is an example of what geodesic paths looks like
        ![](diagrams/ingestion/geodesic_on_an_oblate_ellipsoid.png)  
- <font color=purple>Rows</font> are similar to latitude but do not have uniform spacing. They can be used to determine the position along the path.

<div class="alert-info">
<font color=darkblue>Using these coordinates together, it is possible to determine the latitude and longitude.  However, for consistency and simplicity, and since the alternative is retrieving and parsing a shapefile, we will ping USGS' web tool for path row conversion.  You can see the original tool here: https://landsat.usgs.gov/wrs-2-pathrow-latitudelongitude-converter</font>
    </div>

In [2]:
#Function to turn path/row into lat/lon
def pr_to_ll(path, row, show_url = False):
    """Path/Row WRS coordinates to Latitude and Longitude.
    This function uses USGS' landsat calculator for lat/long conversions via an HTTP request
    """
    
    url = "https://landsat.usgs.gov/landsat/lat_long_converter/tools_latlong.php?rs=convert_pr_to_ll"
    a = [path,row]    
    for i in range(len(a)):
        url = url + "&rsargs[]=" + str(a[i])
    url = url + "&rsrnd=" + str(current_milli_time())
    if(show_url): print("view directly from USGS:", url)
    r = requests.get(url)  
    table = pd.read_html(r.text)[0]
    lat, lon = table[1][1], table[3][1]
    return {"Latitude":float(lat),"Longitude":float(lon)}

## Why Is It So Complicated?
The Euclidean Distance Formula is the method method most people think of when they need to calculate distance but is really only suited to finding the shortest straight line between points because it is based on the Pythagorean Theorem even in higher dimensions.
<font color=darkblue>$$\sqrt{\sum_{i=1}^n (a_{i} - b_{i})^{2}}$$</font>
        - n number of dimensions
        - a point in n-dimensional space
        - b point in n-dimensional space
![](diagrams/ingestion/surface_distance.png) 
The difference between the euclidean distance and the distance traversed across the surface will increase as the points move further apart.  For small applications euclidean distance may be sufficient but when trying to determine the number of pixels to project a satellite image to we need greater accuracy.  For this we use geodesic geometry to calculate the surface distance between points.

![](diagrams/ingestion/geodesic_distance.png)  

Since geodesic distance has a rather intense formula for calculating distance per meter.  We will simplify the equation down to a simplified form after plugging in the values for the WGS84 ellipsoid measures.


 <font color=darkblue>$$"meters-per-degree-latitude" = \frac{\pi}{180}* \frac{\frac{a^{2}*b^{2}}{(\frac{a^{2}*b^{2}}{2})^{\frac{3}{2}}}}{{1+(\frac{a^{2}-b^{2}}{a^{2}+b^{2}}) * cos(2*rad(lat))}^{\frac{3}{2}}} $$</font>

<br>
 
<font color=darkblue>$$"meters-per-degree-longitude" = \frac{a^{2}}{{\frac{a^{2} + b^{2}}{2}}^{\frac{1}{2}}} * \frac{cos(rad(lat))}{{1+(\frac{a^{2}-b^{2}}{a^{2}+b^{2}}) * cos(2*rad(lat))}^{\frac{1}{2}}}$$</font>

- where a and b are the elliptical axes radii from WGS84

In [3]:
#This is the result of using the geodetic distance formula using the
#ellipsoid measurements from WGS84 reduced to constants.
#it should be accurate to within ~1 meter
def meters_per_degree(degrees_latitude):
    """Gets meters per degree of latitude and longitude from latitude"""
    lat_rads = math.radians(degrees_latitude)
    mpd_lat = 111132.92 - 559.82*math.cos(2*lat_rads)+1.175*math.cos(4*lat_rads)
    mpd_lon = 111412.84 * math.cos(lat_rads) - 93.5 * math.cos(3*lat_rads)
    return {"lat_mpd":mpd_lat, "lon_mpd":mpd_lon}

## What Comes Next

Now that we have a way to get the meter distance between degrees of latitude and longitude, we can derive the rest of the pertinent data.  This includes:

   - pixel counts for latitude and longitude
   - resolution per tile size for latitude and longitude
   - tile size in both dimensions
   
Unlike the meters per degree of latitude and longitude, these values are rather straightforward in their derivation.

In [4]:
def pixel_counts(latitude_position, image_resolution = 30):
    """Pixel count information based on the latitude and image resolution.
    This function returns a dictionary of values indicating the pixel counts, tile-sizes,
    resolutions, and estimated meters per degrees for both latitude and longitude.
    """
    
    temp = meters_per_degree(latitude_position)
    lat_pxl_count = math.floor(temp["lat_mpd"]/image_resolution)
    lon_pxl_count = math.floor(temp["lon_mpd"]/image_resolution)
    res_lat = -(image_resolution/temp["lat_mpd"])
    res_lon = image_resolution/temp["lon_mpd"]
    tile_size_lat = -(res_lat*lat_pxl_count)
    tile_size_lon = res_lon*lon_pxl_count
    cond1 = (tile_size_lat / res_lat)%1==0
    cond2 = (tile_size_lon / res_lon)%1==0
    if(cond1 & cond2):
        return {"lat_pxl_count":lat_pxl_count, "lon_pxl_count":lon_pxl_count,
                "resolution_lat":res_lat, "resolution_lon":res_lon, "tile_size_lat":tile_size_lat,
                "tile_size_lon":tile_size_lon, "meters_per_deg_lat":temp["lat_mpd"], "meters_per_deg_lon":temp["lon_mpd"]}
    

## Make Things Convenient

Now that we have a way to get the values we require, we should create wrapper functions to simplify the process.  We will create two wrappers.  on will allow for the input of latitude and longitude and the other will take the path and row.

In [5]:
def pr_to_pixel_counts(path = 0, row = 0, resolution = 30):
    """Path/Row WRS coordinates to pixel count information.
    This function is a wrapper for the pixel_counts function that allows you to pass
    path and row coordinates in order to get the pixel counts, tile-sizes,
    resolutions, and estimated meters per degrees for both latitude and longitude.
    """
    
    lat_lon = pr_to_ll(path,row)
    return {**lat_lon, **pixel_counts(lat_lon["Latitude"])}

def ll_to_pixel_counts(latitude = 0, longitude = 0, resolution = 30):
    """Latitude and Longitude coordinates to pixel count information.
    This function is a wrapper for the pixel_counts function that allows you to pass
    latitude and longitude coordinates in order to get the pixel counts, tile-sizes,
    resolutions, and estimated meters per degrees for the given latitude and longitude.
    """
    
    lat_lon = {"Latitude":latitude,"Longitude":longitude}
    return {**lat_lon, **pixel_counts(lat_lon["Latitude"])}

In [6]:
#in order to get pixel count information from path/row format:
pr_to_pixel_counts(4,5,30) #with path, row, and image_resolution as the arguments

{'Latitude': 77.049,
 'Longitude': -25.208,
 'lat_pxl_count': 3721,
 'lon_pxl_count': 834,
 'meters_per_deg_lat': 111637.22846518604,
 'meters_per_deg_lon': 25028.242043474733,
 'resolution_lat': -0.0002687275599049422,
 'resolution_lon': 0.001198645911602149,
 'tile_size_lat': 0.9999352504062899,
 'tile_size_lon': 0.9996706902761923}

In [7]:
#to get all of the pixel count information from latitude/Longitude format:
ll_to_pixel_counts(77.049,-25.208,30) #only latitude and image resolution matter for the calculations

{'Latitude': 77.049,
 'Longitude': -25.208,
 'lat_pxl_count': 3721,
 'lon_pxl_count': 834,
 'meters_per_deg_lat': 111637.22846518604,
 'meters_per_deg_lon': 25028.242043474733,
 'resolution_lat': -0.0002687275599049422,
 'resolution_lon': 0.001198645911602149,
 'tile_size_lat': 0.9999352504062899,
 'tile_size_lon': 0.9996706902761923}

In [8]:
#in order to get the meters per degree for latitude and longitude directly, use:
meters_per_degree(77.049)

{'lat_mpd': 111637.22846518604, 'lon_mpd': 25028.242043474733}